# My Lesson 1

**Homework tasks**
1. [DONE] Get AWS instance running (either g2 or m4 if not yet approved for g2) after contacting support etc.
2. [DONE] Setup ssh keys as per instructions in setup video
3. [DONE] install bash setup script onto server instance
4. [DONE] launch jupyter notebook on the instance
5. [CURRENT] once the notebook is running, review the lesson 1 notebook notes and run each cell of code to figure out what python and vgg is doing
6. install kaggle CLI onto the server instance
7. use the kaggle CLI to download the current data for the Dogs vs. Cats Redux competition
8. configure the new data to the file structure in the same way that was used in the sample lesson 1 notebook
9. make a copy of the lesson 1 notebook and use the new copy to draw in the new Dogs Vs. Cats data (if you copy the notebook outside of the course folder, don't forget the utils.py, vgg26.py files etc)
10. Run the relevant code cells on the sample set of new Dogs v. Cats data to make a prediction on the new image data set.
11. Once the sample set works, modify the jupyter notebook to use the new test data images
12. write a script that takes the predict() data of the new Dogs vs. Cats data and writes the data to a new csv file in the format of the sample_submission.csv file that was downloaded with the Dogs vs. Cats
13. submit that new submission.csv file to the kaggle via the CLI tool
14. check the public scoreboard for your own ranking
15. modify or tune current code in the lesson 1 notebook to try to get into the top 50% ranking of the current Dogs v Cats competition
16. start exploring the other new datasets on kaggle and decide which one you or some teammates would like to study further during the course
17. download the new data to your EC2 instance and repeat the previous steps with your brand new data.